In [3]:
import pandas as pd


In [3]:
Weather_full = pd.read_csv("./Wetterdaten_Basel_stündlich22_24.csv", skiprows=9)

In [4]:
Weather_full.head()


,timestamp,Basel Temperature [2 m elevation corrected],Basel Precipitation Total,Basel Wind Speed [10 m],Basel Wind Direction [10 m]
0,20220101T0000,9.060529,0.0,7.729527,152.24146
1,20220101T0100,8.900529,0.0,8.287822,145.61966
2,20220101T0200,8.740529,0.0,8.145870,135.00000
3,20220101T0300,8.290529,0.0,7.704336,127.40535
4,20220101T0400,7.980528,0.0,7.729527,117.75854


In [6]:
# Rename the column
Weather_full.rename(columns={'timestamp': 'Datum'}, inplace=True)

# Change the format of the 'Datum' column
Weather_full['Datum'] = pd.to_datetime(Weather_full['Datum'], format='%Y%m%dT%H%M')

# Sort by 'Datum' and set it as the index
Weather_full.sort_values(by='Datum', inplace=True)
Weather_full.set_index('Datum', inplace=True)

# Display the first few rows to verify the changes
Weather_full.head()

,Basel Temperature [2 m elevation corrected],Basel Precipitation Total,Basel Wind Speed [10 m],Basel Wind Direction [10 m]
Datum,,,,
2022-01-01 00:00:00,9.060529,0.0,7.729527,152.24146
2022-01-01 01:00:00,8.900529,0.0,8.287822,145.61966
2022-01-01 02:00:00,8.740529,0.0,8.145870,135.00000
2022-01-01 03:00:00,8.290529,0.0,7.704336,127.40535
2022-01-01 04:00:00,7.980528,0.0,7.729527,117.75854


In [7]:
Weather_full.to_csv("./Cleaned/Wetterdaten_Basel_cleanded22_24.csv")

In [4]:
Gas_full = pd.read_csv("./Gasverbrauch_im_Versorgungsgebiet_der_IWB.csv", sep=';')

In [5]:
Gas_full.head()

,Start der Messung,Gasverbrauch,Datum,Zeit,Jahr,Monat,Tag,Wochentag,Tag des Jahres,Quartal,Woche des Jahres
0,2023-10-18T23:00:00+00:00,185054.4299,2023-10-19,01:00,2023,10,19,3,292,4,42
1,2023-10-19T00:00:00+00:00,184888.9114,2023-10-19,02:00,2023,10,19,3,292,4,42
2,2023-10-19T01:00:00+00:00,196871.6645,2023-10-19,03:00,2023,10,19,3,292,4,42
3,2023-10-19T04:00:00+00:00,322512.8374,2023-10-19,06:00,2023,10,19,3,292,4,42
4,2023-10-19T05:00:00+00:00,348277.5490,2023-10-19,07:00,2023,10,19,3,292,4,42


In [6]:
Gas_full.sort_values(by="Start der Messung", inplace=True)
Gas_full.head()

Gas_full.drop(columns=["Datum", "Zeit", "Jahr", "Monat", "Tag", "Wochentag", "Tag des Jahres", "Quartal", "Woche des Jahres"], inplace=True)

Gas_full['Start der Messung'] = Gas_full['Start der Messung'].str.replace(r'\+00:00', '', regex=True)

Gas_full.rename(columns={'Start der Messung': 'Datum'}, inplace=True)
Gas_full.set_index('Datum', inplace=True)

Gas_full.index = Gas_full.index.str.replace('T', ' ')

In [7]:


Gas_full = Gas_full.loc["2022-01-01 00:00:00":]
Gas_full.tail()


,Gasverbrauch
Datum,
2024-11-21 00:00:00,407054.8689
2024-11-21 01:00:00,406493.6664
2024-11-21 02:00:00,423860.2532
2024-11-21 03:00:00,455408.6505
2024-11-21 04:00:00,571633.8426


In [8]:
Gas_full.to_csv("./Cleaned/Gasverbrauch_cleaned.csv")